<a href="https://colab.research.google.com/github/eelcofolkertsma/BIX-samples/blob/main/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a python script to collect dependencies between types in xml library

In [7]:
#Identify registry holding schema-s and output destination

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file \"{name}\" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))



       # "XML='<?xml  version=\"1.0\" encoding=\"UTF-8\" ?>'\n",
        #"LIBRARY='<xs:schema targetNamespace=\"http://www.iata.org/IATA/2015/00/2025.3/IATA_Baggage_CommonTypes\" elementFormDefault=\"qualified\" version=\

Saving IATA_Baggage_BagRQ.xsd to IATA_Baggage_BagRQ.xsd
User uploaded file "IATA_Baggage_BagRQ.xsd" with length 144308 bytes


In [8]:
#!pip install xmlschema
import xmlschema
for fn in uploaded.keys():
  xsd = xmlschema.XMLSchema(uploaded[fn])

In [ ]:
from google.colab import files

with open(OUT, 'w') as f:
  f.write(xsd)

files.download(OUT)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import xml.etree.ElementTree as ET

# The xmlschema library does not have a direct 'build_sample_document' method.
# We will create a very basic sample XML with just the root element.

if xsd.root_elements:
    # Get the qualified name of the first root element
    root_qname = xsd.root_elements[0].name

    # Create an ElementTree element for the root
    root_element = ET.Element(root_qname)

    # Convert the ElementTree to a pretty-printed XML string
    sample_xml = ET.tostring(root_element, encoding='unicode', xml_declaration=True)

    print(sample_xml)
else:
    print("No root elements found in the schema to generate a sample XML.")

<?xml version='1.0' encoding='utf-8'?>
<ns0:IATA_Baggage_BagRQ xmlns:ns0="http://www.iata.org/IATA/2015/00/2025.4/IATA_Baggage_BagRQ" />


## Identify Mandatory Children

### Subtask:
Analyze the `xsd` (XMLSchema) object to identify the mandatory child elements of the root element. This will involve inspecting the schema definition for required elements and their types.


**Reasoning**:
The subtask requires identifying mandatory child elements of the root element in the `xsd` object. I will access the root element, then iterate through its components to find those with `min_occurs` greater than 0, storing their names.



In [11]:
if xsd.root_elements:
    root_element = xsd.root_elements[0]
    mandatory_children = []

    # Iterate through the components (child elements) of the root element
    # The 'components' attribute typically holds complex type definitions
    # For direct child elements defined within the root element's type, we need to dig a bit deeper.
    # A common pattern is to access the 'content' or 'elements' of the root element's type definition.

    # Check if the root element has a complex type definition
    if hasattr(root_element, 'type') and hasattr(root_element.type, 'content'):
        for content_item in root_element.type.content:
            if hasattr(content_item, 'name') and hasattr(content_item, 'min_occurs') and content_item.min_occurs > 0:
                mandatory_children.append(content_item.name)
            elif hasattr(content_item, 'elements'): # Handle sequences/choices within complex types
                for element_decl in content_item.elements:
                    if element_decl.min_occurs > 0:
                        mandatory_children.append(element_decl.name)

    print(f"Root Element: {root_element.name}")
    if mandatory_children:
        print("Mandatory Child Elements:")
        for child in mandatory_children:
            print(f"- {child}")
    else:
        print("No mandatory child elements found for the root element.")
else:
    print("No root elements found in the schema.")

Root Element: {http://www.iata.org/IATA/2015/00/2025.4/IATA_Baggage_BagRQ}IATA_Baggage_BagRQ
Mandatory Child Elements:
- {http://www.iata.org/IATA/2015/00/2025.4/IATA_Baggage_BagRQ}BagSegment


## Generate XML with Mandatory Children

### Subtask:
Modify the existing code to create an XML document that includes the root element and all its identified mandatory child elements, using `xml.etree.ElementTree`. The children will be added as empty elements.


**Reasoning**:
I need to generate an XML document that includes the root element and all its mandatory child elements, as identified in the previous step. I will use `xml.etree.ElementTree` to construct the XML structure by creating the root element and then appending each mandatory child as an empty sub-element. Finally, I will convert this structure to a pretty-printed XML string and display it.



In [12]:
import xml.etree.ElementTree as ET

# Create an ET.Element for the root_qname
root = ET.Element(root_qname)

# Iterate through mandatory_children and add them as sub-elements
for child_qname in mandatory_children:
    ET.SubElement(root, child_qname)

# Convert the constructed ET.Element tree to a pretty-printed XML string
# ET.tostring itself doesn't pretty print, so we'll use a simple approach for now
# For better pretty printing, xml.dom.minidom could be used, but keeping it simple for the subtask.

# We'll re-implement a simple way to pretty print for demonstration or use the default
# For basic pretty printing without minidom:
from xml.dom import minidom
def prettify(elem):
    """Return a pretty-printed XML string for the Element."""
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ", encoding="utf-8").decode('utf-8')

sample_xml_with_children = prettify(root)

print(sample_xml_with_children)

<?xml version="1.0" encoding="utf-8"?>
<ns0:IATA_Baggage_BagRQ xmlns:ns0="http://www.iata.org/IATA/2015/00/2025.4/IATA_Baggage_BagRQ">
  <ns0:BagSegment/>
</ns0:IATA_Baggage_BagRQ>

